In [ ]:
import pandas as pd
import mysql.connector
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,f1_score
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

%matplotlib inline

In [ ]:
# Connect to SQL Server
conn = mysql.connector.connect(host="localhost", user="root", password="root", database="GooglePlayStore")
# Fetch Data
query = "SELECT * FROM rating_pred"
df = pd.read_sql(query, conn)
conn.close()
df = df.drop('id', axis=1)
# Show first few rows
print(df.head())

C:\Users\shree\AppData\Local\Temp\ipykernel_9980\1944357366.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   Category  Size_in_Mb  Content_Rating  Ad_Supported  In_App_Purchases  \
0         0        10.0               0             0                 0   
1         1         2.9               0             1                 0   
2         2         3.7               0             0                 0   
3         3         1.8               0             1                 0   
4         1         6.2               0             0                 0   

   Transformed_Rating  Installs  Free  Rating_Count  Editors_Choice  
0                   0        10     1             0               0  
1                   4      5000     1            64               0  
2                   0        50     1             0               0  
3                   5        10     1             5               0  
4                   0       100     1             0               0  


In [3]:
df.shape

(2237013, 10)

In [4]:
df.Transformed_Rating.value_counts()

Transformed_Rating
0    1046271
4     561859
5     412277
3     173693
2      38770
1       4143
Name: count, dtype: int64

In [5]:
zero_rating_rows = df[df["Transformed_Rating"] == 0]
rows_to_drop = zero_rating_rows.sample(n=700000, random_state=42)
# Drop these rows from the original DataFrame
df = df.drop(rows_to_drop.index).reset_index(drop=True)
print("New DataFrame Shape:", df.shape)

New DataFrame Shape: (1537013, 10)


In [6]:
df.Transformed_Rating.value_counts()

Transformed_Rating
4    561859
5    412277
0    346271
3    173693
2     38770
1      4143
Name: count, dtype: int64

In [7]:
from imblearn.combine import SMOTETomek
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

# Load dataset
X = df.drop("Transformed_Rating", axis=1)  # Features
Y = df["Transformed_Rating"]  # Target variable

print("🔹 Original Class Distribution:", Counter(Y))
print("New DataFrame Shape:", df.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3, random_state=7,stratify=Y)
smote_tomek = SMOTETomek(random_state=7)
X_train_resampled, Y_train_resampled = smote_tomek.fit_resample(X_train, Y_train)
print("🔹 After SMOTETomek:", Counter(Y_train_resampled))
 
print(f"X shape: {X_train_resampled.shape}, Y shape: {Y_train_resampled.shape}")

🔹 Original Class Distribution: Counter({4: 561859, 5: 412277, 0: 346271, 3: 173693, 2: 38770, 1: 4143})
New DataFrame Shape: (1537013, 10)
🔹 After SMOTETomek: Counter({0: 393291, 1: 388089, 2: 376311, 3: 364145, 5: 353763, 4: 343229})
X shape: (2218828, 9), Y shape: (2218828,)


In [8]:
df.shape

(1537013, 10)

In [9]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, log_loss, roc_auc_score

# Initialize and train the model
hgbc = HistGradientBoostingClassifier(max_iter=100, learning_rate=0.1, random_state=7)
hgbc.fit(X_train_resampled, Y_train_resampled)

# Predictions and probabilities
y_pred = hgbc.predict(X_test)
y_prob = hgbc.predict_proba(X_test)

# Model evaluation
hgbc_ac = accuracy_score(Y_test, y_pred)
print("HistGradientBoosting Classifier Accuracy:", hgbc_ac)

# Classification Report
print("\nClassification Report:")
print(classification_report(Y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred))

# Log Loss
hgbc_log_loss = log_loss(Y_test, y_prob)
print("\nLog Loss:", hgbc_log_loss)

# ROC-AUC Score (only if the problem is binary classification)
if len(np.unique(Y_test)) == 2:
    hgbc_roc_auc = roc_auc_score(Y_test, y_prob[:, 1])
    print("\nROC-AUC Score:", hgbc_roc_auc)

HistGradientBoosting Classifier Accuracy: 0.5701165030014921

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    103881
           1       0.01      0.34      0.02      1243
           2       0.08      0.32      0.12     11631
           3       0.25      0.17      0.20     52108
           4       0.61      0.49      0.54    168558
           5       0.65      0.52      0.57    123683

    accuracy                           0.57    461104
   macro avg       0.43      0.47      0.41    461104
weighted avg       0.65      0.57      0.60    461104


Confusion Matrix:
[[103881      0      0      0      0      0]
 [     0    419    326    120    186    192]
 [     0   2287   3683   1802   2773   1086]
 [     0   7074  11551   8976  19447   5060]
 [     0  16437  22249  19285  82090  28497]
 [     0  14511  10115   5844  29379  63834]]

Log Loss: 0.9883634253012161


In [11]:
from sklearn.linear_model import LogisticRegression

# Initialize and train Logistic Regression
logreg_c = LogisticRegression(max_iter=500, random_state=7, class_weight='balanced')
logreg_c.fit(X_train_resampled, Y_train_resampled)

# Predictions and probabilities for Logistic Regression
logreg_pred = logreg_c.predict(X_test)
logreg_prob = logreg_c.predict_proba(X_test)

logreg_ac = accuracy_score(Y_test, logreg_pred)
print("\nLogistic Regression Accuracy:", logreg_ac)

print("\nClassification Report:")
print(classification_report(Y_test, logreg_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, logreg_pred))

logreg_log_loss = log_loss(Y_test, logreg_prob)
print("\nLog Loss:", logreg_log_loss)

if len(np.unique(Y_test)) == 2:
    logreg_roc_auc = roc_auc_score(Y_test, logreg_prob[:, 1])
    print("\nROC-AUC Score:", logreg_roc_auc)

c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Logistic Regression Accuracy: 0.4201503348485374

Classification Report:


c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.92      0.34      0.50    103881
           1       0.00      0.00      0.00      1243
           2       0.00      0.00      0.00     11631
           3       0.07      0.17      0.10     52108
           4       0.50      0.85      0.63    168558
           5       0.60      0.04      0.08    123683

    accuracy                           0.42    461104
   macro avg       0.35      0.23      0.22    461104
weighted avg       0.56      0.42      0.37    461104


Confusion Matrix:
[[ 35689      0      0  68192      0      0]
 [     3      0      0    335    888     17]
 [     6      0      0   2165   9377     83]
 [    35      0      0   8684  43022    367]
 [   316      0      0  21176 144079   2987]
 [  2734      0      0  22697  92971   5281]]

Log Loss: 1.7221447002161394


In [ ]:
# Initialize and train RandomForestClassifier
rdf_c = RandomForestClassifier(random_state=7)
rdf_c.fit(X_train_resampled, Y_train_resampled)

rdf_pred = rdf_c.predict(X_test)
rdf_prob = rdf_c.predict_proba(X_test)

rdf_ac = accuracy_score(Y_test, rdf_pred)
print("\nRandomForest Accuracy:", rdf_ac)

print("\nClassification Report:")
print(classification_report(Y_test, rdf_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, rdf_pred))

rdf_log_loss = log_loss(Y_test, rdf_prob)
print("\nLog Loss:", rdf_log_loss)

if len(np.unique(Y_test)) == 2:
    rdf_roc_auc = roc_auc_score(Y_test, rdf_prob[:, 1])
    print("\nROC-AUC Score:", rdf_roc_auc)

ValueError: Unknown label type: continuous-multioutput. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
# Initialize and train DecisionTreeClassifier
dtree_c = DecisionTreeClassifier(random_state=7, criterion='entropy', max_depth=10, min_samples_leaf=2, min_samples_split=5)
dtree_c.fit(X_train_resampled, Y_train_resampled)

dtree_pred = dtree_c.predict(X_test)
dtree_prob = dtree_c.predict_proba(X_test)

dtree_ac = accuracy_score(Y_test, dtree_pred)
print("\nDecisionTreeClassifier Accuracy:", dtree_ac)

print("\nClassification Report:")
print(classification_report(Y_test, dtree_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, dtree_pred))

dtree_log_loss = log_loss(Y_test, dtree_prob)
print("\nLog Loss:", dtree_log_loss)

if len(np.unique(Y_test)) == 2:
    dtree_roc_auc = roc_auc_score(Y_test, dtree_prob[:, 1])
    print("\nROC-AUC Score:", dtree_roc_auc)

In [ ]:
# Initialize and train Bernoulli Naive Bayes
NB = BernoulliNB(binarize=0.0)
NB.fit(X_train_resampled, Y_train_resampled)

y_pred = NB.predict(X_test)
y_prob = NB.predict_proba(X_test)

nb_ac = accuracy_score(Y_test, y_pred)
print("\nBernoulli Naive Bayes Accuracy:", nb_ac)

print("\nClassification Report:")
print(classification_report(Y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred))

nb_log_loss = log_loss(Y_test, y_prob)
print("\nLog Loss:", nb_log_loss)

if len(np.unique(Y_test)) == 2:
    nb_roc_auc = roc_auc_score(Y_test, y_prob[:, 1])
    print("\nROC-AUC Score:", nb_roc_auc)

In [ ]:
# Initialize and train XGBoost Classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=7, use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train_resampled, Y_train_resampled)

y_pred = xgb.predict(X_test)
y_prob = xgb.predict_proba(X_test)

xgb_ac = accuracy_score(Y_test, y_pred)
print("\nXGBoost Classifier Accuracy:", xgb_ac)

print("\nClassification Report:")
print(classification_report(Y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred))

xgb_log_loss = log_loss(Y_test, y_prob)
print("\nLog Loss:", xgb_log_loss)

if len(np.unique(Y_test)) == 2:
    xgb_roc_auc = roc_auc_score(Y_test, y_prob[:, 1])
    print("\nROC-AUC Score:", xgb_roc_auc)

In [ ]:
# Initialize and train LightGBM Classifier
lgb_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
lgb_model.fit(X_train_resampled, Y_train_resampled)

y_pred = lgb_model.predict(X_test)
y_prob = lgb_model.predict_proba(X_test)

lgb_ac = accuracy_score(Y_test, y_pred)
print("\nLightGBM Classifier Accuracy:", lgb_ac)

print("\nClassification Report:")
print(classification_report(Y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred))

lgb_log_loss = log_loss(Y_test, y_prob)
print("\nLog Loss:", lgb_log_loss)

if len(np.unique(Y_test)) == 2:
    lgb_roc_auc = roc_auc_score(Y_test, y_prob[:, 1])
    print("\nROC-AUC Score:", lgb_roc_auc)

In [ ]:
# Initialize and train CatBoost Classifier
Cat_Boost = CatBoostClassifier(verbose=300, n_estimators=1000, learning_rate=0.2, depth=10, early_stopping_rounds=300)
Cat_Boost.fit(X_train_resampled, Y_train_resampled)

cb_ac = Cat_Boost.score(X_train_resampled, Y_train_resampled)
print("\nCatBoost Accuracy:", cb_ac)

y_pred = Cat_Boost.predict(X_test)
y_prob = Cat_Boost.predict_proba(X_test)

f1 = f1_score(Y_test, y_pred, average="weighted")
print("\nF1 Score:", f1)

print("\nClassification Report:")
print(classification_report(Y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred)) 

cb_log_loss = log_loss(Y_test, y_prob)
print("\nLog Loss:", cb_log_loss)

if len(np.unique(Y_test)) == 2:
    cb_roc_auc = roc_auc_score(Y_test, y_prob[:, 1])
    print("\nROC-AUC Score:", cb_roc_auc)

# Save model
with open("finetuning_three.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)